### 1. Load a Janelia COSEM Dataset into Xarray

This function, `open_cosem_dataset`, helps load a 3D image volume from the [Janelia COSEM](https://www.janelia.org/project-team/cosem) dataset into an `xarray.Dataset`. It does the following:

- Constructs the full URL from the dataset root and group path.
- Opens a Zarr array using anonymous access via `fsspec`.
- Extracts voxel spacing metadata (in nanometers) and uses it to create physical coordinates (in meters).
- Wraps the data as an `xarray.DataArray` and then into a `Dataset` for ease of use.

The result is a spatially aware dataset, with proper coordinates, ready for visualization or analysis.


In [ ]:
import xarray as xr
import fsspec
import zarr
import numpy as np


def open_cosem_dataset(dataset_url, group_path, var_name="values"):
    """
    Open a COSEM Zarr volume as an xarray.Dataset with physical coordinates.

    Parameters
    ----------
    dataset_url : str
        e.g. "s3://janelia-cosem-datasets/jrc_hela-1/jrc_hela-1.zarr"
    group_path : str
        e.g. "recon-1/em/fibsem-uint8/s4"
    var_name : str
        Name to assign to the variable in the Dataset

    Returns
    -------
    xr.Dataset
    """
    # Open remote store
    full_url = f"{dataset_url}/{group_path}"
    z = zarr.open_array(full_url, mode="r", storage_options={"anon": True})

    # Default dims (Z, Y, X in COSEM, we'll reverse to X, Y, Z for xarray consistency)
    shape = z.shape
    dims = ["z", "y", "x"]
    if len(shape) != 3:
        raise ValueError(f"Expected 3D data, got shape: {shape}")

    # Try to get voxel spacing in nm
    voxel_size_nm = z.attrs.get("pixelResolution", {}).get(
        "dimensions", [4.0, 4.0, 4.0]
    )  # [Z, Y, X]

    # Build coordinate arrays in physical units (meters)
    coords = {}
    for dim, size, spacing_nm in zip(dims, shape, voxel_size_nm):
        coords[dim] = np.arange(size) * spacing_nm * 1e-9  # convert nm → meters

    # Construct DataArray with coords
    da = xr.DataArray(z, dims=dims, coords=coords, attrs=dict(z.attrs))

    # Wrap into a Dataset
    ds = xr.Dataset({var_name: da})
    return ds

### 2. Retrieve the COSEM Dataset

Here, we use the `open_cosem_dataset` function to load a specific FIB-SEM volume from the COSEM dataset collection:

- `url` points to the dataset root in the S3 bucket.
- `group` identifies the subvolume (at a particular resolution level).

The result is assigned to `ds`, an `xarray.Dataset` that includes:

- A single data variable called `"values"`,
- Dimensions named `z`, `y`, and `x`,
- Coordinates in physical space (meters), based on metadata.

In [ ]:
url = "s3://janelia-cosem-datasets/jrc_hela-1/jrc_hela-1.zarr"
group = "recon-1/em/fibsem-uint8/s4"

ds = open_cosem_dataset(url, group)

### 3. Visualize with Pan3D Viewer

We now create a `Pan3D` viewer instance to visualize the dataset.

- The dataset `ds` is passed to `XArrayViewer`, which sets up a Trame-based UI for interactive volume exploration.
- Once `viewer.ui.ready` resolves, the viewer is fully initialized.
- The `viewer.ui` object can be rendered directly in the notebook (if supported), or externally in a browser window.

This viewer makes it easy to inspect and explore 3D microscopy datasets with physical scale preserved.


In [ ]:
# Create the instance of the viewer, and pass the filter to the pipeline argument

from pan3d.viewers.preview import XArrayViewer

viewer = XArrayViewer(xarray=ds, server="preview")
await viewer.ui.ready

viewer.ui